<a href="https://colab.research.google.com/github/PIYAL-DATTA/Multi-attack/blob/main/NSL_KDD_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Precision, Recall

In [ ]:
path = kagglehub.dataset_download("hassan06/nslkdd")

print("Path to dataset files:", path)

train_path = os.path.join(path, "KDDTrain+.txt")
test_path  = os.path.join(path, "KDDTest+.txt")

print("Path to Train dataset files:", train_path)
print("Path to Test dataset files:", test_path)

column_names = [
    'duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
    'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations',
    'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count','serror_rate',
    'srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate',
    'dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
    'dst_host_srv_rerror_rate','label','difficulty_level'
]

train_df = pd.read_csv(train_path, names=column_names)
test_df = pd.read_csv(test_path, names=column_names)
df = pd.concat([train_df, test_df], ignore_index=True)
df

Path to dataset files: /kaggle/input/nslkdd
Path to Train dataset files: /kaggle/input/nslkdd/KDDTrain+.txt
Path to Test dataset files: /kaggle/input/nslkdd/KDDTest+.txt


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.72,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal,21
148513,0,tcp,http,SF,317,938,0,0,0,0,...,1.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal,21
148514,0,tcp,http,SF,54540,8314,0,0,0,2,...,1.00,0.00,0.00,0.00,0.00,0.00,0.07,0.07,back,15
148515,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.99,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal,21


In [ ]:
def map_attack(label):
    dos = ['back','land','neptune','pod','smurf','teardrop','mailbomb','apache2','processtable','udpstorm']
    probe = ['satan','ipsweep','nmap','portsweep','mscan','saint']
    r2l = ['guess_passwd','ftp_write','imap','phf','multihop','warezmaster','warezclient','spy','xlock','xsnoop','snmpguess','snmpgetattack','httptunnel','sendmail','named']
    u2r = ['buffer_overflow','loadmodule','rootkit','perl','sqlattack','xterm','ps']

    if label == 'normal': return 0 # 'normal'
    elif label in dos: return 1 # 'DoS'
    elif label in probe: return 2 # 'Probe'
    elif label in r2l: return 2 # 'R2L'
    elif label in u2r: return 2 # 'U2R'
    else: return 2 # 'unknown'
# 2 represent BotNet attack.
# 'Probe' Early stage, 'R2L' and 'U2R' attack phase

df['attack_class'] = df['label'].apply(map_attack)
df

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level,attack_class
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20,0
1,0,udp,other,SF,146,0,0,0,0,0,...,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15,0
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19,1
3,0,tcp,http,SF,232,8153,0,0,0,0,...,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21,0
4,0,tcp,http,SF,199,420,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148512,0,tcp,smtp,SF,794,333,0,0,0,0,...,0.06,0.01,0.01,0.01,0.00,0.00,0.00,normal,21,0
148513,0,tcp,http,SF,317,938,0,0,0,0,...,0.00,0.01,0.01,0.01,0.00,0.00,0.00,normal,21,0
148514,0,tcp,http,SF,54540,8314,0,0,0,2,...,0.00,0.00,0.00,0.00,0.00,0.07,0.07,back,15,1
148515,0,udp,domain_u,SF,42,42,0,0,0,0,...,0.01,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,0


# 1D CNN model

In [ ]:
# Assuming df is your loaded NSL-KDD dataset

# 1. Handle categorical features (protocol_type, service, flag)
label_encoders = {}
categorical_cols = ['protocol_type', 'service', 'flag', 'attack_class']  # adjust as needed

for col in categorical_cols:
    if col in df.columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

# 2. Separate features and target
X = df.drop(['attack_class', 'difficulty_level', 'label'], axis=1, errors='ignore')
y = df['attack_class']

# 3. Normalize numerical features
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# 4. Convert target to one-hot encoding for multi-class classification
num_classes = len(np.unique(y))
y_categorical = to_categorical(y, num_classes=num_classes)

# 5. Reshape data for CNN (samples, timesteps, features)
X_reshaped = X.reshape(X.shape[0], X.shape[1], 1)

# 6. Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_reshaped, y_categorical, test_size=0.2, random_state=42)

# 7. Build 1D CNN model for multi-class classification
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Custom metrics
precision = Precision()
recall = Recall()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', precision, recall])

# 8. Train the model
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=32,
                    validation_data=(X_test, y_test))

# 9. Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# 10. Generate classification report
print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, digits=4, zero_division=0))

# 11. Calculate F1-Score (not directly available in Keras metrics)
from sklearn.metrics import f1_score
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
print(f"\nWeighted F1-Score: {f1:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 24s 5ms/step - accuracy: 0.9236 - loss: 0.2230 - precision: 0.9345 - recall: 0.9076 - val_accuracy: 0.9576 - val_loss: 0.1089 - val_precision: 0.9627 - val_recall: 0.9535
Epoch 2/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.9596 - loss: 0.1109 - precision: 0.9621 - recall: 0.9573 - val_accuracy: 0.9675 - val_loss: 0.0912 - val_precision: 0.9684 - val_recall: 0.9670
Epoch 3/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - accuracy: 0.9664 - loss: 0.0909 - precision: 0.9681 - recall: 0.9648 - val_accuracy: 0.9788 - val_loss: 0.0615 - val_precision: 0.9791 - val_recall: 0.9778
Epoch 4/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 22s 4ms/step - accuracy: 0.9736 - loss: 0.0721 - precision: 0.9751 - recall: 0.9721 - val_accuracy: 0.9770 - val_loss: 0.0596 - val_precision: 0.9771 - val_recall: 0.9767
Epoch 5/20
3713/3713 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.9768 - loss: 0.0611 - precision: 0.9778 - recall: 0.9755 - val_accuracy: 0.9819